# IMDB数据集，评论情感二分类微调

1. 完成一个简单的模型下游任务训练过程
2. 简单了解hf 一些常用库的用法


# 加载数据集
使用 datasets 库加载 imdb 数据集。这个库会自动下载并缓存数据。
也可以下载到本地，这样可以防止网络问题导致代码执行失败（尽管已经下载到缓存）


In [3]:
from datasets import load_dataset

dataset_name = "/data/Dataset/LLM-dataset/imdb" # 替换为本地目录，或者使用"imdb"自动下载
raw_dataset = load_dataset(dataset_name)

Generating train split: 25000 examples [00:00, 152530.13 examples/s]
Generating test split: 25000 examples [00:00, 169834.63 examples/s]


这里简单介绍一下数据集加载函数
- load_dataset(path, name=None, data_dir=None, data_files=None, split=None, cache_dir=None, download_mode=None, ignore_verifications=False, keep_in_memory=False, features=None, **kwargs)
  - 功能:  这是最核心的函数，用于从 Hugging Face Hub、本地文件或自定义脚本加载数据集。
  - 参数:
    - path:  数据集的名称 (例如 “rotten_tomatoes”, “glue”, “csv”, “json”) 或本地文件路径。
    - name:  数据集的配置名称 (如果数据集有多个配置，例如 “glue” 数据集的 “cola”, “sst2” 等)。
    - data_dir:  本地数据目录 (如果数据集存储在本地)。
    - data_files:  指定要加载的数据文件 (例如 ["train.csv", "test.csv"])。  可以是字符串或字符串列表。
    - split:  指定要加载的数据集分割 (例如 “train”, “validation”, “test”, “train[:10%]” 表示训练集的前 10%)。  可以是字符串或字符串列表。
    - cache_dir:  指定数据集缓存目录。
    - download_mode:  指定下载模式 (例如 “reuse_dataset_if_exists”, “force_redownload”)。
    - ignore_verifications:  是否忽略数据集校验。
    - keep_in_memory:  是否将数据集保存在内存中。
    - features:  指定数据集的特征 (例如 Features({"text": Value("string"), "label": ClassLabel(names=["neg", "pos"])}))。
    - **kwargs:  传递给数据集构建脚本的其他参数。
  - 返回值:  一个 Dataset 对象 (如果只加载一个分割) 或一个 DatasetDict 对象 (如果加载多个分割)。

In [4]:
'''可以直接print 看一下数据集的概况'''
print(raw_dataset)

'''由于返回的是train 和 test 两个split 也可以直接像字典一样进行索引'''
print(type(raw_dataset['train']))
print(raw_dataset['train'])

print(raw_dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})
<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking poli

# 数据预处理 (Tokenization)
接着我们需要加载 模型对应的分词器对数据进行处理

In [5]:
from transformers import AutoTokenizer

model_checkpoint = "/data/Weights/bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def preprocess(instance):
    return tokenizer(instance["text"], truncation=True, padding="max_length", max_length=160) 

res = preprocess(raw_dataset["train"][0])
print(res)

{'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 1010, 2016, 2038, 102], 'token_type_ids': [0,

数据集处理函数
- map(function, batched=False, num_proc=None, remove_columns=None, keep_in_memory=False, load_from_cache_file=True, cache_file_name=None, writer_batch_size=1000, features=None, disable_nullable=False, desc=None)
  - 功能:  将一个函数应用于数据集的每个样本。  这是最常用的数据处理函数。
  - 参数:
    - function:  要应用于每个样本的函数。  该函数应该接受一个字典作为输入 (表示一个样本)，并返回一个字典作为输出 (表示处理后的样本)。
    - batched:  是否以批处理模式应用函数。  如果为 True，则函数将接收一个字典列表作为输入 (表示一批样本)。
    - num_proc:  用于并行处理的进程数。
    - remove_columns:  要从数据集中删除的列的名称列表。
    - keep_in_memory:  是否将处理后的数据集保存在内存中。
    - load_from_cache_file:  是否从缓存文件加载处理后的数据集。
    - cache_file_name:  缓存文件的名称。
    - writer_batch_size:  写入缓存文件的批处理大小。
    - features:  指定处理后的数据集的特征。
    - disable_nullable:  是否禁用可空列。
    - desc:  进度条的描述。
  - 返回值:  一个处理后的 Dataset 对象。


In [6]:
tokenized_dataset = raw_dataset.map(preprocess, batched=True)

Map: 100%|██████████| 25000/25000 [00:04<00:00, 5308.21 examples/s]


1. 由于hf 中Trainer的一些特殊规定，数据集的标签key值规定为 “labels”，所以我们需要修改数据集中的colnum names；
2. 此外，由于我们已经使用分词器得到token id 所以text数据已经没用了，可以remove掉

In [7]:
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label","labels")
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
})

In [8]:
# 设置数据集格式为 PyTorch tensors (如果使用 TensorFlow 则设置为 "tf")
tokenized_dataset.set_format("torch")

#  加载预训练模型
加载带有适合下游任务头部的模型。对于文本分类，我们使用 AutoModelForSequenceClassification。其就是在最后加了层MLP

In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels=num_labels)
print(model)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /data/Weights/bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

#  定义评估指标
选择适合任务的评估指标。对于分类任务，常用准确率 (Accuracy)、F1 分数等。

这里要注意evaluate.load是要联网下载代码的，注意网络问题（会卡很久）。当然对于一些简单的指标计算也可以自己写，但要注意以下要求

compute_metrics (Callable[[EvalPrediction], Dict], optional) 
- 用于在评估时计算指标的函数。
- 必须接受一个EvalPrediction并返回一个字典，其中键为字符串，值为指标值。
- 注意：当传入的TrainingArgs中batch_eval_metrics设置为True时，您的compute_metrics函数必须接受一个布尔类型的compute_result参数。
- 这将在最后一个评估批次后触发，以通知函数需要计算并返回全局汇总统计信息，而不是累积批次级别的统计信息。

**class transformers.EvalPrediction**

( predictions: typing.Union[numpy.ndarray, tuple[numpy.ndarray]]label_ids: typing.Union[numpy.ndarray, tuple[numpy.ndarray]]inputs: typing.Union[numpy.ndarray, tuple[numpy.ndarray], NoneType] = Nonelosses: typing.Union[numpy.ndarray, tuple[numpy.ndarray], NoneType] = None )

Parameters:

- `predictions` (np.ndarray) — Predictions of the model.
- `label_ids` (np.ndarray) — Targets to be matched.
- `inputs` (np.ndarray, optional) — Input data passed to the model.
- `losses` (np.ndarray, optional) — Loss values computed during evaluation.

Evaluation output (always contains labels), to be used to compute metrics.

In [10]:
metric_path = "/data/cache/hf/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14/accuracy.py"

metric = evaluate.load(metric_path) 
metric

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

这里简单实现了一个指标计算的函数

In [11]:
def compute(references, predictions):

    n_references = len(references)
    n_predictions = len(predictions)

    if n_references != n_predictions:
        raise ValueError("Input lists 'references' and 'predictions' must have the same length.")

    correct_count = 0
    for i in range(n_references):
        if references[i] == predictions[i]:
            correct_count += 1

    accuracy = correct_count / n_references

    return {'accuracy': accuracy}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return compute(predictions=predictions, references=labels)

# 模型训练
在最后会详细的列出TrainingArguments的所有参数并，附上每个参数的意义


In [ ]:
training_args = TrainingArguments(
    output_dir="./results/bert-epoch5",              # 输出目录，保存模型和日志
    eval_strategy="epoch",         # 每个 epoch 结束后进行评估
    save_strategy="epoch",               # 每个 epoch 结束后保存模型
    learning_rate=2e-5,                  # 学习率
    per_device_train_batch_size=16,      # 训练批次大小
    per_device_eval_batch_size=16,       # 评估批次大小
    num_train_epochs=5,                  # 训练轮数
    weight_decay=0.01,                   # 权重衰减
    load_best_model_at_end=True,         # 训练结束后加载最佳模型
    metric_for_best_model="accuracy",    # 以 accuracy 指标判断最佳模型 (需与 compute_metrics 返回的 key 匹配)
    report_to="tensorboard",             # 可以选择保存日志到 tensorboard, wandb 等
    save_total_limit=2,
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

[2025-05-04 22:54:46,393] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/data/anaconda3/envs/llm/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


In [15]:
trainer.train()

/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.342100,0.241491,0.899640
2,0.180400,0.255857,0.901880
3,0.113100,0.347929,0.899880
4,0.056700,0.433937,0.899120
5,0.037600,0.481276,0.899200


/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/anaconda3/envs/llm/lib/python3.10/site

TrainOutput(global_step=3910, training_loss=0.13811987757377917, metrics={'train_runtime': 1018.8572, 'train_samples_per_second': 122.686, 'train_steps_per_second': 3.838, 'total_flos': 1.02777756e+16, 'train_loss': 0.13811987757377917, 'epoch': 5.0})

我们查看训练日志可以发现，非常明显的过拟合现象，train—loss下降，eval-loss上升。

大概率是因为进行了模型的全量微调，且数据集过小导致的。那么我们简单尝试一下只微调部分层（pooler层和classifier）

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels=num_labels)

for name, param in model.named_parameters():
    if 'classifier' in name or 'pooler' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False
        
# 检查一下模型可训练参数

total_params = 0
trainable_params = 0
for name, param in model.named_parameters():
    total_params += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
        print(f"- {name}")

print(f"\nTotal parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Percentage trainable: {trainable_params / total_params * 100:.2f}%")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /data/Weights/bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


- bert.pooler.dense.weight
- bert.pooler.dense.bias
- classifier.weight
- classifier.bias

Total parameters: 109483778
Trainable parameters: 592130
Percentage trainable: 0.54%


In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.602900,0.481677,0.786480
2,0.470900,0.441038,0.800040
3,0.465500,0.427341,0.806680
4,0.449700,0.423719,0.807800
5,0.443400,0.422067,0.808640


/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/anaconda3/envs/llm/lib/python3.10/site

TrainOutput(global_step=3910, training_loss=0.4802803332238551, metrics={'train_runtime': 867.0357, 'train_samples_per_second': 144.169, 'train_steps_per_second': 4.51, 'total_flos': 1.02777756e+16, 'train_loss': 0.4802803332238551, 'epoch': 5.0})

可以发现至少训练是正常的，可以逐步增加可训练的参数，或者使用peft库进行高效微调

# 模型推理

这里使用hf 的pipeline来进行模型的推理。对于使用标准 hf 训练得到的模型权重，直接传入目录即可。如果是经过其他包装的模型需要重新定义模型

In [16]:
from transformers import pipeline

custom_pipeline = pipeline(
    "text-classification",  # 任务类型
    model="./results/bert-epoch5/checkpoint-3910",
    tokenizer=tokenizer
)

Device set to use cuda:0


In [17]:
# 使用pipeline进行预测
result = custom_pipeline(["I hate this movie","i love this movie"])

def postprocess(res):
    Res_dict = {"LABEL_1":"positive", "LABEL_0":"negative"}
    
    return Res_dict[res['label']]


presult = list(map(postprocess, result))

print(presult)

['negative', 'positive']


# HuggingFace TrainingArguments 参数解析


## 1. 基础配置参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| output_dir | str | "trainer_output" | 输出目录，用于保存模型预测结果和检查点 |
| overwrite_output_dir | bool | False | 是否覆盖输出目录的内容，用于在已有检查点的目录中继续训练 |
| do_train | bool | False | 是否运行训练，主要由训练/评估脚本使用 |
| do_eval | bool | None | 是否在验证集上运行评估，当eval_strategy不为"no"时会自动设为True |
| do_predict | bool | False | 是否在测试集上运行预测，主要由训练/评估脚本使用 |
| run_name | str | 等同于output_dir | 运行描述符，通常用于wandb、mlflow和comet日志记录 |
| disable_tqdm | bool | None | 是否禁用进度条，默认在日志级别为warn或更低时为True |
| remove_unused_columns | bool | True | 是否自动移除模型前向方法中未使用的列 |
| seed | int | 42 | 训练开始时设置的随机种子 |
| data_seed | int | None | 数据采样器使用的随机种子，若未设置则使用与seed相同的值 |
| use_cpu | bool | False | 是否使用CPU。若为False，会优先使用CUDA或MPS设备 |
| full_determinism | bool | False | 若为True，则调用enable_full_determinism而非set_seed以确保分布式训练中的可复现性 |

## 2. 训练控制参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| per_device_train_batch_size | int | 8 | 每个GPU/TPU核心/CPU的训练批量大小 |
| per_device_eval_batch_size | int | 8 | 每个GPU/TPU核心/CPU的评估批量大小 |
| gradient_accumulation_steps | int | 1 | 在执行反向/更新传递之前累积梯度的更新步骤数 |
| eval_accumulation_steps | int | None | 在将结果移至CPU之前累积输出张量的预测步骤数 |
| max_steps | int | -1 | 若为正数，表示要执行的训练步骤总数，会覆盖num_train_epochs |
| num_train_epochs | float | 3.0 | 要执行的训练周期总数 |
| max_grad_norm | float | 1.0 | 最大梯度范数（用于梯度裁剪） |
| group_by_length | bool | False | 是否将训练数据集中大致相同长度的样本分组（提高效率） |
| length_column_name | str | "length" | 预计算长度的列名 |
| dataloader_drop_last | bool | False | 如果数据集长度不能被批量大小整除，是否丢弃最后一个不完整批次 |
| dataloader_num_workers | int | 0 | 用于数据加载的子进程数量 |
| dataloader_pin_memory | bool | True | 是否在数据加载器中使用内存锁定 |
| dataloader_persistent_workers | bool | False | 数据集消耗完后是否保持工作进程活跃 |
| dataloader_prefetch_factor | int | None | 每个工作进程提前加载的批次数 |
| auto_find_batch_size | bool | False | 是否通过指数衰减自动寻找适合内存的批量大小 |
| neftune_noise_alpha | float | None | 激活NEFTune噪声嵌入的参数，可显著提高指令微调的模型性能 |

## 3. 评估相关参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| eval_strategy | str/IntervalStrategy | "no" | 训练期间的评估策略："no"(不评估)、"steps"(每eval_steps评估)、"epoch"(每个epoch结束时评估) |
| prediction_loss_only | bool | False | 在执行评估和生成预测时，是否仅返回损失 |
| eval_steps | int/float | None | 如果eval_strategy="steps"，则为两次评估之间的更新步骤数，未设置则与logging_steps相同 |
| eval_delay | float | None | 首次评估前等待的轮次或步骤数，取决于eval_strategy |
| load_best_model_at_end | bool | False | 训练结束时是否加载找到的最佳模型 |
| metric_for_best_model | str | None | 与load_best_model_at_end结合使用，指定用于比较不同模型的指标 |
| greater_is_better | bool | None | 与load_best_model_at_end和metric_for_best_model结合使用，指定更好的模型是否应具有更大的指标 |
| include_inputs_for_metrics | bool | False | 已弃用，使用include_for_metrics替代 |
| include_for_metrics | List[str] | [] | 指定metrics计算时需要的额外数据，如"inputs"、"loss"等 |
| eval_do_concat_batches | bool | True | 是否在批次间递归连接输入/损失/标签/预测结果 |
| batch_eval_metrics | bool | False | 是否在每个批次结束时调用compute_metrics以累积统计信息，而非将所有评估logits保存在内存中 |
| eval_on_start | bool | False | 是否在训练前执行评估步骤（检查验证步骤是否正常工作） |
| eval_use_gather_object | bool | False | 是否从所有设备递归收集嵌套列表/元组/字典中的对象 |

## 4. 优化器与学习率相关参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| learning_rate | float | 5e-5 | AdamW优化器的初始学习率 |
| weight_decay | float | 0 | 在AdamW优化器中应用于所有层的权重衰减（除了所有偏置和LayerNorm权重） |
| adam_beta1 | float | 0.9 | AdamW优化器的beta1超参数 |
| adam_beta2 | float | 0.999 | AdamW优化器的beta2超参数 |
| adam_epsilon | float | 1e-8 | AdamW优化器的epsilon超参数 |
| lr_scheduler_type | str/SchedulerType | "linear" | 要使用的调度器类型 |
| lr_scheduler_kwargs | dict | {} | 传递给lr_scheduler的额外参数 |
| warmup_ratio | float | 0.0 | 用于从0到learning_rate的线性预热的总训练步骤的比例 |
| warmup_steps | int | 0 | 用于从0到learning_rate的线性预热的步骤数，会覆盖warmup_ratio的效果 |
| optim | str/OptimizerNames | "adamw_torch" | 要使用的优化器，如"adamw_hf"、"adamw_torch"、"adamw_torch_fused"等 |
| optim_args | str | None | 传递给优化器的可选参数 |
| optim_target_modules | str/List[str] | None | 要优化的目标模块，目前用于GaLore和APOLLO算法 |

## 5. 保存与加载相关参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| save_strategy | str/SaveStrategy | "steps" | 训练期间的检查点保存策略："no"(不保存)、"epoch"(每个epoch末保存)、"steps"(每save_steps保存)、"best"(在达到新的best_metric时保存) |
| save_steps | int/float | 500 | 若save_strategy="steps"，则为两次检查点保存之间的更新步骤数 |
| save_total_limit | int | None | 如果传递了值，将限制检查点的总数，删除output_dir中较旧的检查点 |
| save_safetensors | bool | True | 使用safetensors而非默认torch.load和torch.save进行状态字典的保存和加载 |
| save_on_each_node | bool | False | 在多节点分布式训练中，是否在每个节点上保存模型和检查点 |
| save_only_model | bool | False | 检查点时是否只保存模型，而不保存优化器、调度器和RNG状态 |
| restore_callback_states_from_checkpoint | bool | False | 是否从检查点恢复回调状态 |
| resume_from_checkpoint | str | None | 指向包含有效检查点的文件夹路径，用于恢复训练 |
| ignore_data_skip | bool | False | 恢复训练时，是否跳过数据到之前训练的相同阶段 |

## 6. 分布式训练参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| local_rank | int | -1 | 分布式训练期间的进程排名 |
| ddp_backend | str | None | 分布式训练使用的后端，必须是"nccl"、"mpi"、"ccl"、"gloo"、"hccl"之一 |
| tpu_num_cores | int | None | 在TPU上训练时的TPU核心数量 |
| ddp_find_unused_parameters | bool | None | 使用分布式训练时传递给DistributedDataParallel的find_unused_parameters标志的值 |
| ddp_bucket_cap_mb | int | None | 使用分布式训练时传递给DistributedDataParallel的bucket_cap_mb标志的值 |
| ddp_broadcast_buffers | bool | None | 使用分布式训练时传递给DistributedDataParallel的broadcast_buffers标志的值 |
| ddp_timeout | int | 1800 | torch.distributed.init_process_group调用的超时设置 |
| fsdp | bool/str/List[FSDPOption] | "" | 使用PyTorch分布式并行训练（仅在分布式训练中） |
| fsdp_config | str/dict | None | 与FSDP一起使用的配置 |
| deepspeed | str/dict | None | 使用Deepspeed，实验性功能 |
| accelerator_config | str/dict/AcceleratorConfig | None | 用于内部Accelerator实现的配置 |
| split_batches | bool | None | 加速器是否应该将数据加载器生成的批次在设备间拆分 |
| average_tokens_across_devices | bool | False | 是否跨设备平均令牌数，用于精确损失计算 |

## 7. 精度与性能参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| bf16 | bool | False | 是否使用bf16 16位（混合）精度训练而非32位训练 |
| fp16 | bool | False | 是否使用fp16 16位（混合）精度训练而非32位训练 |
| fp16_opt_level | str | "O1" | 用于fp16训练的Apex AMP优化级别 |
| fp16_backend | str | "auto" | 已弃用，使用half_precision_backend替代 |
| half_precision_backend | str | "auto" | 用于混合精度训练的后端 |
| bf16_full_eval | bool | False | 是否使用完全的bfloat16评估而非32位 |
| fp16_full_eval | bool | False | 是否使用完全的float16评估而非32位 |
| tf32 | bool | None | 是否启用TF32模式，在Ampere及更新的GPU架构中可用 |
| jit_mode_eval | bool | False | 是否使用PyTorch jit trace进行推理 |
| use_ipex | bool | False | 在可用时是否使用Intel PyTorch扩展 |
| torch_compile | bool | False | 是否使用PyTorch 2.0的torch.compile编译模型 |
| torch_compile_backend | str | None | 在torch.compile中使用的后端 |
| torch_compile_mode | str | None | 在torch.compile中使用的模式 |
| gradient_checkpointing | bool | False | 是否使用梯度检查点以节省内存（以较慢的反向传递为代价） |
| gradient_checkpointing_kwargs | dict | None | 传递给gradient_checkpointing_enable方法的关键字参数 |
| torch_empty_cache_steps | int | None | 在调用torch.<device>.empty_cache()之前等待的步骤数 |
| use_liger_kernel | bool | False | 是否启用Liger Kernel进行LLM模型训练，可提高多GPU训练吞吐量并减少内存使用 |

## 8. 日志与报告参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| log_level | str | "passive" | 主进程上使用的日志级别："debug"、"info"、"warning"、"error"、"critical"或"passive" |
| log_level_replica | str | "warning" | 副本上使用的日志级别 |
| log_on_each_node | bool | True | 在多节点分布式训练中，是否在每个节点上记录日志 |
| logging_dir | str | None | TensorBoard日志目录，默认为output_dir/runs/CURRENT_DATETIME_HOSTNAME |
| logging_strategy | str/IntervalStrategy | "steps" | 训练期间的日志策略："no"、"epoch"或"steps" |
| logging_first_step | bool | False | 是否记录第一个global_step |
| logging_steps | int/float | 500 | 如果logging_strategy="steps"，则为两次日志之间的更新步骤数 |
| logging_nan_inf_filter | bool | True | 是否过滤日志记录的nan和inf损失 |
| report_to | str/List[str] | "all" | 报告结果和日志的集成列表，支持多种平台如wandb、tensorboard等 |
| skip_memory_metrics | bool | True | 是否跳过向指标添加内存分析器报告 |
| include_tokens_per_second | bool | None | 是否计算每秒每设备令牌数的训练速度指标 |
| include_num_input_tokens_seen | bool | None | 是否跟踪整个训练过程中看到的输入令牌数 |

## 9. 集成与推送参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| push_to_hub | bool | False | 是否在每次保存模型时将模型推送到Hub |
| hub_model_id | str | None | 与本地output_dir保持同步的仓库名称 |
| hub_strategy | str/HubStrategy | "every_save" | 定义何时以及什么内容被推送到Hub |
| hub_token | str | None | 用于将模型推送到Hub的令牌 |
| hub_private_repo | bool | None | 是否将仓库设为私有 |
| hub_always_push | bool | False | 是否在上一次推送未完成时跳过推送检查点 |

## 10. 其他工具性参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| debug | str/List[DebugOption] | "" | 启用一个或多个调试功能，实验性功能 |
| past_index | int | -1 | 一些模型可以利用过去的隐藏状态进行预测 |
| ray_scope | str | "last" | 使用Ray进行超参数搜索时使用的范围 |
| label_names | List[str] | None | 输入字典中对应标签的键列表 |
| label_smoothing_factor | float | 0.0 | 标签平滑因子 |
| torchdynamo | str | None | TorchDynamo的后端编译器，如"eager"、"inductor"等 |
| use_mps_device | bool | False | 已弃用。若可用，将使用mps设备（类似于cuda设备） |


# HuggingFace Trainer 参数解析


## 1. 模型相关参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| model | PreTrainedModel 或 torch.nn.Module | 必选 | 用于训练、评估或预测的模型。如果未提供，则必须提供model_init |
| model_init | Callable[[], PreTrainedModel] | None | 实例化模型的函数。如果提供，每次调用train将从此函数返回的新模型实例开始 |
| place_model_on_device | bool | True | 是否自动将模型放置在设备上。当使用模型并行或DeepSpeed时会设为False |

## 2. 数据处理参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| data_collator | DataCollator | None | 将数据集元素列表组合成批次的函数。如果未提供，且无processing_class，将默认使用default_data_collator |
| train_dataset | Dataset或IterableDataset | None | 用于训练的数据集。如果是datasets.Dataset，不被model.forward()接受的列会自动移除 |
| eval_dataset | Dataset或Dict[str, Dataset] | None | 用于评估的数据集。如果是字典，将对每个数据集进行评估，并在指标名称前加上字典键 |
| processing_class | PreTrainedTokenizerBase等 | None | 用于处理数据的类。如果提供，将用于自动处理模型输入，并与模型一起保存 |

## 3. 训练配置参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| args | TrainingArguments | 基本实例 | 用于调整训练的参数。如果未提供，将默认使用output_dir设置为当前目录中tmp_trainer的基本实例 |
| compute_loss_func | Callable | None | 接受模型原始输出、标签和累积批次中项目总数的函数，并返回损失 |

## 4. 优化器和调度器参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| optimizers | Tuple[Optimizer, LRScheduler] | (None, None) | 包含优化器和调度器的元组。默认使用AdamW和get_linear_schedule_with_warmup |
| optimizer_cls_and_kwargs | Tuple[Type[Optimizer], Dict[str, Any]] | None | 包含优化器类和关键字参数的元组。覆盖args中的optim和optim_args，与optimizers参数不兼容 |

## 5. 评估和指标参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| compute_metrics | Callable[[EvalPrediction], Dict] | None | 用于计算评估指标的函数。必须接受EvalPrediction并返回字符串到指标值的字典 |
| preprocess_logits_for_metrics | Callable | None | 在每个评估步骤缓存logits之前预处理它们的函数。必须接受两个张量(logits和labels)并返回处理后的logits |

## 6. 回调参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|-----|-------|------|
| callbacks | List[TrainerCallback] | None | 用于自定义训练循环的回调列表。会添加到默认回调列表中 |

## 7. 重要属性

| 属性名 | 类型 | 说明 |
|-------|-----|------|
| model | PreTrainedModel | 始终指向核心模型。如果使用transformers模型，它将是PreTrainedModel的子类 |
| model_wrapped | Module | 始终指向最外层的模型。如果原始模型被一个或多个其他模块包装，这是应该用于前向传播的模型 |
| is_model_parallel | bool | 模型是否已切换到模型并行模式(与数据并行不同，这意味着模型的某些层在不同的GPU上拆分) |
| is_in_train | bool | 模型当前是否正在运行train(例如，在train期间调用evaluate时) |



# Hugging Face Tokenizer 方法和属性全解析

## 1. 核心方法

| 方法名 | 说明 | 常用参数 |
|-------|------|---------|
| `__call__` | 主要的tokenization方法，支持单条或多条文本 | `text`, `text_pair`, `padding`, `truncation`, `max_length`, `return_tensors` |
| `encode` | 将文本转换为token IDs | `text`, `text_pair`, `add_special_tokens`, `padding`, `truncation` |
| `decode` | 将token IDs转换回文本 | `token_ids`, `skip_special_tokens`, `clean_up_tokenization_spaces` |
| `tokenize` | 将文本分割为tokens | `text`, `text_pair`, `add_special_tokens` |
| `batch_encode_plus` | 批量处理文本(与`__call__(batch_text)`相同) | `batch_text_or_text_pairs`, `padding`, `truncation`, `max_length` |
| `encode_plus` | 获取文本的完整编码信息 | 与`__call__`相同 |
| `save_pretrained` | 保存tokenizer到目录 | `save_directory`, `legacy_format`, `filename_prefix` |
| `from_pretrained` | 从预训练模型或目录加载tokenizer(静态方法) | `pretrained_model_name_or_path`, `cache_dir`, `use_fast` |

## 2. 转换方法

| 方法名 | 说明 | 常用参数 |
|-------|------|---------|
| `convert_tokens_to_ids` | 将tokens转换为IDs | `tokens` |
| `convert_ids_to_tokens` | 将IDs转换为tokens | `ids`, `skip_special_tokens` |
| `convert_tokens_to_string` | 将tokens列表转换为单个字符串 | `tokens` |
| `prepare_for_model` | 为模型准备编码输入 | `ids`, `pair_ids`, `add_special_tokens` |
| `build_inputs_with_special_tokens` | 在输入中添加特殊tokens | `token_ids_0`, `token_ids_1` |

## 3. 辅助方法

| 方法名 | 说明 | 常用参数 |
|-------|------|---------|
| `pad` | 对已编码的输入执行填充 | `encoded_inputs`, `padding_strategy`, `max_length`, `pad_to_multiple_of` |
| `truncate` | 截断已编码的输入 | `encoded_inputs`, `max_length`, `truncation_strategy` |
| `add_special_tokens` | 处理特殊tokens | `encoded_inputs`, `return_tensors` |
| `create_token_type_ids_from_sequences` | 为序列对创建token类型IDs | `token_ids_0`, `token_ids_1` |
| `num_special_tokens_to_add` | 计算添加的特殊tokens数量 | `pair` |

## 4. 关键属性

| 属性名 | 类型 | 说明 |
|-------|------|------|
| `vocab_size` | int | 词汇表大小 |
| `model_max_length` | int | 模型支持的最大序列长度 |
| `padding_side` | str | 填充位置("left"或"right") |
| `truncation_side` | str | 截断位置("left"或"right") |
| `all_special_tokens` | list | 所有特殊tokens列表 |
| `special_tokens_map` | dict | 特殊token类型到值的映射 |
| `vocab` | dict | 词汇表(token到ID的映射) |
| `added_tokens_encoder` | dict | 额外添加的tokens编码器 |
| `added_tokens_decoder` | dict | 额外添加的tokens解码器 |

## 5. `__call__`方法主要参数

| 参数名 | 类型 | 默认值 | 说明 |
|-------|------|-------|------|
| `text` | str/List[str] | 必需 | 要编码的文本或文本列表 |
| `text_pair` | str/List[str] | None | 配对文本(用于句子对任务) |
| `padding` | bool/str | False | 填充策略("max_length"、"longest"或布尔值) |
| `truncation` | bool/str | False | 截断策略("only_first"、"only_second"、"longest_first"、"do_not_truncate"或布尔值) |
| `max_length` | int | None | 序列的最大长度 |
| `stride` | int | 0 | 当使用`return_overflowing_tokens=True`时的窗口滑动步长 |
| `return_tensors` | str | None | 返回的张量类型("pt"表示PyTorch，"tf"表示TensorFlow，"np"表示NumPy) |
| `return_token_type_ids` | bool | None | 是否返回token类型IDs |
| `return_attention_mask` | bool | None | 是否返回注意力掩码 |
| `return_overflowing_tokens` | bool | False | 是否返回溢出tokens |
| `return_special_tokens_mask` | bool | False | 是否返回特殊token掩码 |
| `return_offsets_mapping` | bool | False | 是否返回字符偏移映射 |
| `return_length` | bool | False | 是否返回tokens长度 |
| `verbose` | bool | True | 是否打印警告信息 |

